# Selenium
allows us to access a web browser.  
Especially required for webpages run on JavaScript

1. Install selenium if you don't have it:

    pip install selenium
    

2. download the relevant brower for use within your script path:

    https://github.com/mozilla/geckodriver/releases  
    https://sites.google.com/a/chromium.org/chromedriver/downloads

## Download current rainfall data:
Meteorological Service Singapore Rainfall observations

In [ ]:
url = 'http://www.weather.gov.sg/weather-currentobservations-rainfall'

In [ ]:
from bs4 import BeautifulSoup as bs
import urllib.request as r
from selenium import webdriver
import pandas as pd
import re

require your *web driver* in folder:- 
 - and put in path as below

In [ ]:
driver = webdriver.Firefox(executable_path='../../WebDriver/geckodriver')

Retrieve the page:

In [ ]:
driver.get(url)

In [ ]:
html30mins = driver.page_source
bsObj = bs(html30mins,"lxml")

Get data timestamp:

In [ ]:
element = bsObj.find("img", {"id":"basemap"})
# time pattern
pattern_time = re.compile('[0-9]+:[0-9]+')
timestamp = pattern_time.findall(element.attrs['src'])[0]
timestamp

In [ ]:
# Location regex pattern
pattern_loc = re.compile('<strong>(.*)</strong>')

In [ ]:
# Prepare dataframe
df = pd.DataFrame(columns = ['Time', 'Station Id', 'Location', '30mins', '1hour', '2hours', 'Daily'])

Get the default 30 mins rainfall data:

In [ ]:
dataset30mins =  bsObj.findAll("",{"class":"sgr"})

Get 1 hour data

In [ ]:
driver.find_element_by_id('1hour').click()

In [ ]:
html1hour = driver.page_source
bsObj = bs(html1hour,"lxml")
dataset1hour =  bsObj.findAll("",{"class":"sgr"})

Get 2 hour data

In [ ]:
driver.find_element_by_id('2hours').click()

In [ ]:
html2hours = driver.page_source
bsObj = bs(html2hours,"lxml")
dataset2hours =  bsObj.findAll("",{"class":"sgr"})

Get today's rainfall:

In [ ]:
driver.find_element_by_id('last').click()

In [ ]:
htmlToday = driver.page_source
bsObj = bs(html2hours,"lxml")
datasetToday =  bsObj.findAll("",{"class":"sgr"})

Put them together:

In [ ]:
i = 0
for data in dataset30mins:
    df = df.append({'Time':timestamp, 
                    'Station Id':data.get('id'), 
                    'Location':pattern_loc.search(data.get('data-content')).group(1),
                    '30mins':data.get_text(), 
                    '1hour':dataset1hour[i].get_text(), 
                    '2hours':dataset2hours[i].get_text(),
                    'Daily':datasetToday[i].get_text()}, 
                   ignore_index=True)
    i+=1

In [ ]:
df.head()

#### References:
http://stanford.edu/~mgorkove/cgi-bin/rpython_tutorials/Scraping_a_Webpage_Rendered_by_Javascript_Using_Python.php  
http://selenium-python.readthedocs.io/  